In [ ]:
import os

print(os.getcwd())
os.chdir('D:/code/HAP/BehaveMine')
print(os.getcwd())

In [ ]:
import importlib
import pandas as pd
import pickle

import src.exp as exp
import utils

importlib.reload(exp)
importlib.reload(utils)

In [49]:
dataset = 'hh103'

exp_dir = os.getcwd()

with open(f'{exp_dir}/data/{dataset}/exp_data.pkl', 'rb') as f:
    exp_data = pickle.load(f)

with open(f'{exp_dir}/data/{dataset}/data_map.pkl', 'rb') as f:
    data_map = pickle.load(f)


def time_stage(time):
    if 2 <= time < 6:
        return 0
    elif 6 <= time < 10:
        return 1
    elif 10 <= time < 14:
        return 2
    elif 14 <= time < 18:
        return 3
    elif 18 <= time < 22:
        return 4
    else:
        return 5


sensor_len = len(data_map['sensor_map'])
length1 = []
length2 = []

for i in range(len(exp_data)):
    item = exp_data[i]
    item['time_stage'] = time_stage(item['start_time'].hour)

    item['sensor_happen_id_list'] = item['sensor_happen_id_list'][-50:]
    item['sensor_between_id_list'] = item['sensor_between_id_list'][-50:]
    if len(item['sensor_between_id_list']) <= 0:
        item['sensor_between_id_list'] = item['sensor_happen_id_list'][-1:]
    length1.append(len(item['sensor_happen_id_list']))
    length2.append(len(item['sensor_between_id_list']))

length1 = pd.Series(length1)
length2 = pd.Series(length2)

train_data, test_data, eval_data = utils.split_data(exp_data)

In [45]:
importlib.reload(exp)
importlib.reload(utils)

# parameters
epoch = 100
lr = 0.001
batch_size = 64
model_params = {
    'input_dim': [len(data_map['activity_map']), 6, len(data_map['location_map']),
                  len(data_map['sensor_map'])],
    'b_hidden_dim': [64, 4, 4], 's_hidden_dim': [64, 4, 4]}
window_size = 6

In [ ]:
print(f'Training...')
exp.train(train_data, test_data, epoch, lr, batch_size,
          window_size, model_params, dataset)

In [ ]:
print(f'Evaluating...')
eval_result = exp.eval(eval_data, window_size, model_params, dataset)
accuracy, macro_f1 = utils.cal_accuracy(eval_result, len(data_map['activity_map']))
print(accuracy)
print(macro_f1)